In [1]:
# 라이브러리
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

c:\Users\leesa\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# gpu 사용가능 여부 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# 시드 고정
torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)

cuda


In [3]:
# 학습에 쓰일 파라미터 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 64

In [4]:
# 데이터셋 다운
train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                    train=True, # True를 지정하면 훈련 데이터로 다운로드
                    transform=transforms.ToTensor(), # 텐서로 변환
                    download=True)

test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                   train=False, # False를 지정하면 테스트 데이터로 다운로드
                   transform=transforms.ToTensor(), # 텐서로 변환
                   download=True)

In [5]:
# 데이터의 배치 사이즈 정의
data_loader = torch.utils.data.DataLoader(dataset=train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [8]:
# 클래스 생성
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        # 첫번째 합성곱 + 활성화 함수 + 맥스 풀링
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # 두번째 합성곱 + 활성화 함수 + 맥스 풀링
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # 전결합층 : 10개의 출력
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias = True)

        # 전결합층의 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) # 전결합층 들어가기 전에 flatten
        out = self.fc(out)
        return out

In [9]:
# 모델 선언
model = CNN().to(device)

In [10]:
# 옵티마이저와 손실함수
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# 배치 수
total_batch = len(data_loader)
total_batch

937

In [12]:
for epoch in range(training_epochs):

    avg_loss = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        Y_pred = model(X)
        loss = criterion(Y_pred, Y)
        loss.backward()
        optimizer.step()

        avg_loss += loss / total_batch

In [13]:
# 테스트
with torch.no_grad():
    X_test = test.test_data.view(len(test), 1, 28, 28).float().to(device)
    Y_test = test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

c:\Users\leesa\anaconda3\envs\tf\lib\site-packages\torchvision\datasets\mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
c:\Users\leesa\anaconda3\envs\tf\lib\site-packages\torchvision\datasets\mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9846999645233154


#### 새로운 모델

In [6]:
import warnings
warnings.filterwarnings('ignore')

# gpu 사용 가능 여부 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)

print(device)

# 학습파라미터
learning_rate = 0.001
training_epochs = 10
batch_size = 100

# 데이터셋 정의
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

# 데이터로더로 배치사이즈 정의
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

cuda


In [8]:
# 클래스로 모델 생성
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5

        # 첫번째 합성곱 + 활성화 함수 + 맥스 풀링
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
            )

        # 두번째 합성곱 + 활성화 함수 + 맥스 풀링
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
            )

        # 세번째 합성곱 + 활성화 함수 + 맥스 풀링
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
            )

        # 첫번째 전결합층
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)

        # 전결합층 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc1.weight)

        # 네번째 합성곱 + 활성화 함수 + 맥스 풀링
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob)
            )

        # 두번째 전결합층
        self.fc2 = torch.nn.Linear(625, 10, bias=True)

        # 두번째 전결합층 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [9]:
# 모델정의
model = CNN().to(device)

In [10]:
# 옵티마이저, 손실함수
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# 총 배치
total_batch = len(data_loader)

In [12]:
from tqdm import tqdm
import time

In [13]:
for epoch in range(training_epochs):

    with tqdm(data_loader, unit="batch") as tepoch:

        avg_loss = 0

        for X, Y in data_loader:
            X = X.to(device)
            Y = Y.to(device)

            optimizer.zero_grad()
            Y_pred = model(X)
            loss = criterion(Y_pred, Y)
            loss.backward()
            optimizer.step()

            avg_loss += loss / total_batch

  0%|          | 0/600 [00:14<?, ?batch/s]


KeyboardInterrupt: 

In [21]:
# 학습 결과 확인
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    pred = model(X_test)
    correct_pred = torch.argmax(pred, 1) == Y_test
    accuracy = correct_pred.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9840999841690063


Pytorch와 Tensorflow 모두 GPU 잘 작동!